In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [4]:
loader = PyPDFDirectoryLoader('./C:/Users/shett/OneDrive/Desktop/Project/chatbotuni/scrapped_data')
documentData = loader.load ()
documentData[:60]

[Document(page_content="Scraped Data\nVidyavardhini's College Of Engineering & Technology, Vasai Road\n??????????????? ???????????? ??? ?????????? ???????????, ??? ???\n(Approved by AICTE, DTE Maharashtra and Affiliated to University of Mumbai)\nNBA & NAAC Accredited\nAbout Alumni\nHome»About alumni\nVCET Alumni Association\nAn alumni meet is organized regularly. Alumni?s are contacted through various social media like\nface book, WhatsApp etc. and also invited through personal emails. The date of alumni meet is\nfloated on institute website the alumni?s are appealed to register online for the same. The meet\ncomprises of a welcome program, Interaction, entertainment programs like singing, dancing followed\nby dinner. A feedback is also taken during this occasion. There is an interactive session between\nfaculty and alumni wherein their views and suggestions are noted for betterment of the college. This\ninteraction improves industry contacts for placement, project and internship. The 

### Creating Vectors

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
textSplitter = RecursiveCharacterTextSplitter (
    chunk_size = 1000,
    chunk_overlap = 20
)

In [7]:
refinedDocument = textSplitter.split_documents (documents = documentData)

In [8]:
# refinedDocument

### Storing vectors in form of embeddings in Vector DB

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

In [9]:
vectorDatabase = FAISS.from_documents (
    documents = refinedDocument[:60],
    embedding = OllamaEmbeddings (model = 'llama3')
)

In [10]:
vectorDatabase

In [11]:
query = 'sequence transduction models'
result = vectorDatabase.similarity_search (query=query)
result[0].page_content

'has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head\nattention and the parameter-free position representation and became the other person involved in nearly every\ndetail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and\ntensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and\nefficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and\nimplementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating\nour research.\n†Work performed while at Google Brain.\n‡Work performed while at Google Research.\n31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.arXiv:1706.03762v7  [cs.CL]  2 Aug 2023'

In [12]:
from langchain_community.llms import Ollama
model = Ollama (model = 'llama3')
model

Ollama(model='llama3')

### Designing Prompt Template

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
prompt = ChatPromptTemplate.from_template (
    '''
        Answer the following question based on the provided context.
        <context>
            {context}
        </context>
        Questions : {input}
    '''
)

### Creating the Retriever

In [15]:
retriever = vectorDatabase.as_retriever ()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000149A42E1A90>)

### Creating the Chain for Retriever

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [17]:
documentChain = create_stuff_documents_chain (
    llm = model,
    prompt = prompt
)

### Retrieval Chain

In [18]:
from langchain.chains import create_retrieval_chain

In [19]:
retrievalChain = create_retrieval_chain (
    retriever,
    documentChain
)

In [20]:
response = retrievalChain.invoke ({
    'input' : 'With what are the best performing models connected'
})

In [ ]:
response['answer']